In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/ch5. 소득예측/원본 데이터/train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/ch5. 소득예측/원본 데이터/test.csv')

- 순서상 5번째이지만 합치기 전에 미리 처리해도 될 것 같아서 앞 순서로 처리

In [ ]:
# # Income_Status - Unknown 처리하기 > Over : 5.27% Under: 94.73% 비율에 따라 Unknow비율 나누고 -> Train 상위 317개
# # train_df 처리
# in_con1 = train_df['Income_Status'] == 'Unknown'

# print(train_df[in_con1]['Income_Status'].count())

# # 상위 n번째 해당하는 Income 찾기
# print(train_df[in_con1].sort_values(by = 'Income', ascending = False)[317:318]['Income'])

# # 조건 만들기 : 찾은 값 기준으로 나누기
# in_con2 = train_df[in_con1]['Income'] >= 1744
# in_con3 = train_df[in_con1]['Income'] < 1744

# # 조건에 따라 값 대입하기
# train_df.loc[in_con1 & in_con2, 'Income_Status'] = 'Over Median'
# train_df.loc[in_con1 & in_con3, 'Income_Status'] = 'Under Median'

# # check
# print(train_df.loc[in_con1]['Income_Status'])

- Train & Test 합치기 : 한 번에 전처리하기 위함

In [ ]:
# df = pd.concat([train_df, test_df], ignore_index=True)

In [ ]:
# df.info()

1. Hispanic_Origin 칼럼 value 값의 공백 제거

In [ ]:
df['Hispanic_Origin'] = df['Hispanic_Origin'].str.strip()

In [ ]:
#check
df[df['Hispanic_Origin'] == 'NA'].head(1)

,ID,Age,Gender,Education_Status,Employment_Status,Working_Week (Yearly),Industry_Status,Occupation_Status,Race,Hispanic_Origin,...,Citizenship,Birth_Country,Birth_Country (Father),Birth_Country (Mother),Tax_Status,Gains,Losses,Dividends,Income_Status,Income
629,TRAIN_00629,37,F,Associates degree (Vocational),Children or Armed Forces,0,Agriculture,Farming & Forestry & Fishing,White,NA,...,Native,US,US,US,Married Filling Jointly both under 65 (MFJ),0,0,0,Unknown,0


2. Histonic_Origin 결측치 ('Na') 처리하기   
    -> Birth_Country(출생지)에 따라 Histonic_Origin 최빈값으로 대체

- Hispanic_Origin == NA 이면서 birth_Country != US에 있는 나라들 확인
  >US - 과반수 All other    
  Germany - 과반수 All other    
  Philippines - 과반수 All other    
  Mexico -> 과반수 Mexican (Mexicano)    
  Cuba -> 과반수 Cuban    
  Nicaragua -> 과반수 Central or South American    
  Unknown -> 과반수 All other    
  Canada / Italy -> All other    
    
- Hispanic_origin == Do not know 경우    
  >출생지가 전부 US이며 - 과반수 ALL other

In [ ]:
#df.loc[(df['Hispanic_Origin'] == 'NA') & (df['Birth_Country'] == 'US'),]['Hispanic_Origin'].replace('NA','All other', inplace = True)
# df.loc[행 범위, 바꿀 칼럼] = 넣을 값
# All other -> 'US', 'Germany','Philippines','Unknown','Canada','Italy','Jamaica'
# Meican -> 'Mexico
# Cuban -> 'Cuba
# Central or South American -> Nicaragua

# Do not know - NA로 간주

# All other 경우
for i in ['US', 'Germany','Philippines','Unknown','Canada','Italy','Jamaica'] :
    his_con1 = df['Hispanic_Origin'] == 'NA'
    his_con2 = df['Birth_Country'] == i
    df.loc[his_con1 & his_con2,'Hispanic_Origin'] = 'All other'

# Mexico
his_con1 = df['Hispanic_Origin'] == 'NA'
his_con2 = df['Birth_Country'] == 'Mexico'
df.loc[his_con1 & his_con2,'Hispanic_Origin'] = 'Mexican (Mexicano)'

# Cuba
his_con2 = df['Birth_Country'] == 'Cuba'
df.loc[his_con1 & his_con2,'Hispanic_Origin'] = 'Cuban'

# Nicaragua
his_con2 = df['Birth_Country'] == 'Nicaragua'
df.loc[his_con1 & his_con2,'Hispanic_Origin'] = 'Central or South American'

# Do not know
his_con1 = df['Hispanic_Origin'] == 'Do not know'
df.loc[his_con1,'Hispanic_Origin'] = 'All other'

In [ ]:
# check
df[df['Hispanic_Origin'] == 'NA'].count()

ID                        0
Age                       0
Gender                    0
Education_Status          0
Employment_Status         0
Working_Week (Yearly)     0
Industry_Status           0
Occupation_Status         0
Race                      0
Hispanic_Origin           0
Martial_Status            0
Household_Status          0
Household_Summary         0
Citizenship               0
Birth_Country             0
Birth_Country (Father)    0
Birth_Country (Mother)    0
Tax_Status                0
Gains                     0
Losses                    0
Dividends                 0
Income_Status             0
Income                    0
dtype: int64

3. Household_Status 칼럼 삭제

In [ ]:
del df['Household_Status']

In [ ]:
# check
df.columns

Index(['ID', 'Age', 'Gender', 'Education_Status', 'Employment_Status',
       'Working_Week (Yearly)', 'Industry_Status', 'Occupation_Status', 'Race',
       'Hispanic_Origin', 'Martial_Status', 'Household_Summary', 'Citizenship',
       'Birth_Country', 'Birth_Country (Father)', 'Birth_Country (Mother)',
       'Tax_Status', 'Gains', 'Losses', 'Dividends', 'Income_Status',
       'Income'],
      dtype='object')

In [ ]:
# hispanic & birth전처리 후 저장
# df.to_csv('/content/drive/MyDrive/ch5. 소득예측/train_df_ox.csv',index = False)

6. Birth_Country - Unknown 처리     
  1) 부/모 국적이 같거나 둘 중 하나만 채워진 경우 : 그대로 사용    
  2) 부/모 둘다 Unknown인 경우 :  Hispanic_Origin 기준으로 각 인종마다의 최빈값으로 대체
  *주의) 조건이 겹치거나 누락되지 않도록 주의하기

In [ ]:
# Unknown인 전체 개수
birth_con1 = df['Birth_Country'] == 'Unknown'

df[birth_con1]['ID'].count()

330

In [ ]:
# 1) 부모 국적이 채워진 경우
# -> 아빠의 국적만 있는 경우
birth_con2 = (df['Birth_Country (Father)'] != 'Unknown') & (df['Birth_Country (Mother)'] == 'Unknown')

df[(birth_con1) & (birth_con2)]['Birth_Country'].count()

18

In [ ]:
df.loc[(birth_con1) & (birth_con2),'Birth_Country'] = df['Birth_Country (Father)']

In [ ]:
# check
df[(birth_con1) & (birth_con2)]['Birth_Country'].count()
# 실행 후 바로 적용이 안 되고 시간이 좀 지나서 실행하면 0으로 바뀜

18

In [ ]:
# -> 엄마의 국적만 채워진 경우
birth_con3 = (df['Birth_Country (Father)'] == 'Unknown') & (df['Birth_Country (Mother)'] != 'Unknown')
df.loc[birth_con1 & birth_con3, 'Birth_Country'].count()

10

In [ ]:
df.loc[birth_con1 & birth_con3, 'Birth_Country'] = df['Birth_Country (Mother)']

In [ ]:
# check
df.loc[birth_con1 & birth_con3, 'Birth_Country'].count()

10

In [ ]:
# -> 둘다 채워진 경우
birth_con4 = (df['Birth_Country (Father)'] != 'Unknown') & (df['Birth_Country (Mother)'] != 'Unknown')

df.loc[birth_con1 & birth_con4, 'Birth_Country'].count()

33

In [ ]:
df.loc[birth_con1 & birth_con4, 'Birth_Country'] = df['Birth_Country (Father)']

In [ ]:
# check
df.loc[birth_con1 & birth_con4, 'Birth_Country'].count()

33

In [ ]:
# 2) 부모 국적 모두 Unkown인 경우
birth_con5 = (df['Birth_Country (Father)'] == 'Unknown') & (df['Birth_Country (Mother)'] == 'Unknown')
df[(birth_con1) & (birth_con5)]['Birth_Country'].count()

269

In [ ]:
# Hispanic 별로 가장 많은 개수를 가진 나라 추출하기
a = list(df['Hispanic_Origin'].unique())
dic = {}
for i in  a:
    dic[i] = list(df[df['Hispanic_Origin'] == i].groupby('Birth_Country')['Income'].agg(['count']).sort_values(by = 'count', ascending = False).iloc[:1].index)

In [ ]:
dic

{'All other': ['US'],
 'Cuban': ['Cuba'],
 'Mexican (Mexicano)': ['Mexico'],
 'Central or South American': ['US'],
 'Puerto Rican': ['US'],
 'Mexican-American': ['US'],
 'Other Spanish': ['US'],
 'Do not know': ['US'],
 'Chicano': ['US'],
 'NA': ['US']}

In [ ]:
# 각 hispanic에 따라 출생지 입력하기
for i, j in dic.items():
  con1 = (df['Hispanic_Origin'] == i)
  df.loc[birth_con1 & birth_con5 & con1, 'Birth_Country'] = j[0]

In [ ]:
# check
df.loc[birth_con1 & birth_con5]['Birth_Country'].count()

269

In [ ]:
# 전체적인 Check
df[df['Birth_Country'] == 'Unknown']['ID'].count()

0

523 - 26 - 15 - 44 - 438 = 0 완료

4. Birth_Country Father & Mother → 상관관계가 높으므로 삭제

In [ ]:
del df['Birth_Country (Father)']
del df['Birth_Country (Mother)']

In [ ]:
df.head()

,ID,Age,Gender,Education_Status,Employment_Status,Working_Week (Yearly),Industry_Status,Occupation_Status,Race,Hispanic_Origin,Martial_Status,Household_Summary,Citizenship,Birth_Country,Tax_Status,Gains,Losses,Dividends,Income_Status,Income
0,TRAIN_00000,63,M,Middle (7-8),Full-Time,4,Social Services,Services,White,All other,Married,Householder,Native,US,Nonfiler,0,0,0,Unknown,425
1,TRAIN_00001,37,M,Associates degree (Vocational),Full-Time,52,Entertainment,Services,White,All other,Separated,Householder,Native,US,Single,0,0,0,Under Median,0
2,TRAIN_00002,58,F,High graduate,Full-Time,52,Manufacturing (Non-durable),Admin Support (include Clerical),Black,All other,Married,Householder,Native,US,Married Filling Jointly both under 65 (MFJ),3411,0,0,Under Median,860
3,TRAIN_00003,44,M,High graduate,Full-Time,52,Retail,Technicians & Support,White,All other,Divorced,Householder,Native,US,Single,0,0,0,Under Median,850
4,TRAIN_00004,37,F,High graduate,Full-Time,52,Retail,Sales,White,All other,Divorced,Householder,Native,US,Head of Household (HOH),0,0,0,Unknown,570


In [ ]:
# hispanic 전처리 후 저장
# df.to_csv('/content/drive/MyDrive/ch5. 소득예측/train_df_oo.csv',index = False)

In [ ]:
# bith 전처리 후 저장
# df.to_csv('/content/drive/MyDrive/ch5. 소득예측/train_df_xo.csv',index = False)

In [ ]:
# historic & birth 전처리 안 하고 저장
df.to_csv('/content/drive/MyDrive/ch5. 소득예측/train_df_xx.csv',index = False)

5. Income_Status - Unknown 처리 : Income 상위 317개는 over_median으로, 나머지 5708개는 under_median으로 대체 (기존 데이터 비율에 맞추어 대입)   
    -> 어떻게 배분할지에 대한 고민

In [ ]:
# 정규표현식 잠시 사용
#import re

#in_con2 = []
#for i in df['ID']:
#  in_con2.append(re.compile('TRAIN[0-9]*').match(i))

In [ ]:
# in_con2
#사용 불가능..

In [ ]:
df['ID'].head()

0    TRAIN_00000
1    TRAIN_00001
2    TRAIN_00002
3    TRAIN_00003
4    TRAIN_00004
Name: ID, dtype: object

In [ ]:
df['ID'].isin(['TRAIN_00000'])

0         True
1        False
2        False
3        False
4        False
         ...  
29995    False
29996    False
29997    False
29998    False
29999    False
Name: ID, Length: 30000, dtype: bool

7. 로그 변환 Gains, Losses, Dividends

In [ ]:
# 수치형 변수 로그변환
import numpy as np

columns_to_log = ['Gains', 'Losses','Dividends']

for column in columns_to_log:
    df[column] = np.log1p(df[column])

In [ ]:
# check
df[columns_to_log].describe()

,Gains,Losses,Dividends
count,20000.000000,20000.000000,20000.000000
mean,0.335391,0.161557,0.546952
std,1.672807,1.088791,1.738389
min,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000
max,11.512925,8.379539,10.714440


In [ ]:
df[['Gains', 'Losses','Dividends']].max()

Gains        11.512925
Losses        8.379539
Dividends    10.714440
dtype: float64

- 연속형 변수 / 범주형 변수 모델링 과정 전 칼럼 추출

In [ ]:
# dtype이 int임에도 범주형 변수일 수도 있기에 확인하고 데이터 타입 바꿔주기
# -> 여기서는 전처리가 된 상태로 할 필요가 없다
# df['column명'] = df['column명'].astype(dtype)

df_numeric = df.select_dtypes(include=['int']).columns
df_categorical = df.select_dtypes(include=['object']).columns
# 식별자 ID 제외하기
df_categorical = df_categorical.drop('ID')

In [ ]:
# 연속형 변수 스케일링을 한다면,
# StandardScaler
# MinMaxScaler 아파트 실거래가격 예측과 같은 회귀 문제를 해결할 때 더 적합한 스케일링 기법
# RobustScaler

8. 라벨인코딩 : 범주형 변수 모두 (ID제외)

In [ ]:
from sklearn.preprocessing import LabelEncoder

# # 라벨 인코딩 적용
label_encoder = LabelEncoder()

df[df_categorical] = df[df_categorical].apply(label_encoder.fit_transform)

In [ ]:
# check
df[df_categorical].head()

,Gender,Education_Status,Employment_Status,Industry_Status,Occupation_Status,Race,Hispanic_Origin,Martial_Status,Household_Summary,Citizenship,Birth_Country,Tax_Status,Income_Status
0,1,15,2,20,11,4,0,1,4,2,39,4,1
1,1,1,2,6,11,4,0,4,4,2,39,5,1
2,0,12,2,11,0,1,0,1,4,2,39,2,1
3,1,12,2,19,12,4,0,0,4,2,39,5,1
4,0,12,2,19,10,4,0,0,4,2,39,0,1


9. Train / Test SET 분리

In [ ]:
train_data = df.dropna(subset=['Income'])
test_data = df[df['Income'].isnull()]
test_data = test_data.drop(columns=['Income'])

In [ ]:
# check
print(train_data.shape)
print(test_data.shape)

(20000, 20)
(10000, 19)


In [ ]:
# 전처리한 데이터 저장하기
df.to_csv('/content/drive/MyDrive/ch5. 소득예측/train_df_oo.csv',index = False)
# test_df.to_csv('/content/drive/MyDrive/ch5. 소득예측/test_df.csv',index = False)

확인